# ✅ generate_at_docs_V4 优化版
- 支持新字段 `参数json` → 自动构建 YAML 中的 `参数`
- CSV 输入中应有字段：`参数json`，格式如 `{ "<name>": {"__desc__": "desc", ...}, ... }`

In [ ]:
!pip install pandas jinja2 ruamel.yaml sphinx sphinx-rtd-theme

In [ ]:
import os
import pandas as pd
import json
from ruamel.yaml import YAML
from jinja2 import Environment

In [ ]:
# 读取 CSV
CSV_INPUT = 'at_commands_v5.csv'  # 替换为你自己的文件名
df = pd.read_csv(CSV_INPUT, encoding='utf-8')

def parse_paramjson(raw):
    try:
        obj = json.loads(raw)
        out = []
        for key, val in obj.items():
            desc = val.get('__desc__', '')
            valmap = {k: v for k, v in val.items() if k != '__desc__'}
            out.append({"name": key, "desc": desc, "valmap": valmap})
        return out
    except Exception as e:
        print(f"解析参数json失败: {e}\n{raw}")
        return []

df['参数_parsed'] = df['参数json'].apply(parse_paramjson)

In [ ]:
# 写入 YAML 文件
yaml = YAML()
yaml.default_flow_style = False

OUT_YAML_DIR = 'yaml_output'
os.makedirs(OUT_YAML_DIR, exist_ok=True)

for idx, row in df.iterrows():
    obj = {
        '命令': row['命令'],
        '命令标题': row['命令标题'],
        '命令类型': row['命令类型'],
        '响应': row['响应'],
        '示例命令': row['示例命令'],
        '功能描述': row['功能描述'],
        '备注': row['备注'],
        '参数': row['参数_parsed']
    }
    with open(os.path.join(OUT_YAML_DIR, f"{row['命令']}.yaml"), 'w', encoding='utf-8') as f:
        yaml.dump(obj, f)

print("✅ YAML 输出完成")